# Optimisation of commerical building energy system

ToDo: provide context of decarbonisation task, building energy system model, etc.

#### Setup

In [1]:
# import packages and required functions
from tqdm.auto import tqdm
import numpy as np
from numpy import random
from scipy import stats
from models import run_model
from functools import partial
from utils import get_Gurobi_WLS_env, fmt_design_results

In [ ]:
# quality of life stuff
env = get_Gurobi_WLS_env(silence=True)
run_model = partial(run_model, env=env)
random.seed(42)

In [ ]:
solar_years = range(2012,2018)
load_years = range(2012,2018)

#### Designing using our judgement

In [4]:
# define design options
design_options = [
    [500,400],
    [500,600],
    [600,600],
    [750,600],
    [750,800],
]

Evaluate cost of each design option using the building energy system model

In [5]:
det_results = [run_model(*design) for design in tqdm(design_options, desc='Assessing designs')]

Assessing designs:   0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
print([round(r['capex']/1e3,1) for r in det_results])
print([round(r['opex']/1e3,1) for r in det_results])
print([round(r['total']/1e3,1) for r in det_results])

[65.5, 79.5, 87.0, 98.2, 112.2]
[100.6, 91.4, 76.1, 54.1, 46.3]
[166.1, 170.9, 163.1, 152.3, 158.6]


In [7]:
# find the best design, i.e. the one with the lowest total cost
best_det_design = det_results[np.argmin([r['total'] for r in det_results])]
print(fmt_design_results(best_det_design))

Parameter         Unit      Value
----------------  ------  -------
Solar capacity    kWp       750
Battery capacity  kWh       600
Total cost        £k/yr     152.3
CAPEX             £k/yr      98.2
OPEX              £k/yr      54.1


In [8]:
# find cost of system without solar panels and batteries
nosystem_results = run_model(0, 0)
print(fmt_design_results(nosystem_results))

Parameter         Unit      Value
----------------  ------  -------
Solar capacity    kWp         0
Battery capacity  kWh         0
Total cost        £k/yr     219.2
CAPEX             £k/yr       0
OPEX              £k/yr     219.2


In [9]:
print("Using solar-battery system reduces cost of supplying building load by "\
      f"£{(nosystem_results['total'] - best_det_design['total'])/1e3:,.1f}k/yr "\
      f"({round((nosystem_results['total'] - best_det_design['total']) / nosystem_results['total'] * 100, 1)})%")

Using solar-battery system reduces cost of supplying building load by £66.8k/yr (30.5)%


#### Designing using the Linear Program model

In [10]:
LP_results = run_model(None,None)
print(fmt_design_results(LP_results))

Parameter         Unit      Value
----------------  ------  -------
Solar capacity    kWp       888.7
Battery capacity  kWh       501
Total cost        £k/yr     141
CAPEX             £k/yr     101.7
OPEX              £k/yr      39.3


In [11]:
design_improvement = best_det_design['total'] - LP_results['total']
print("Using the LP for design reduces costs by "\
      f"£{(design_improvement)/1e3:,.1f}k/yr "\
      f"({round((design_improvement) / best_det_design['total'] * 100, 1)})%")

Using the LP for design reduces costs by £11.3k/yr (7.4)%


#### Accounting for uncertainty during design

Initially let's only consider uncertainty in the solar generation. We'll model this uncertainty via the year of solar generation data used in the simulation. Let's assume that every year of data we have available in our dataset (2012 to 2017) is equally likely to occur in the future

In [12]:
stoch_results = [run_model(*design, solar_year=solar_years) for design in tqdm(design_options, desc='Assessing designs')]
# note, we can run our model with different scenarios (each year of solar data) using this nice compact notation
# this is equivalent to doing the folloowing for each design,
design_costs = []
for syear in solar_years:
    design_costs.append(run_model(
        design_options[0][0],
        design_options[0][1],
        solar_year=[syear]
        )['total'])
avg_cost = np.mean(design_costs)

Assessing designs:   0%|          | 0/5 [00:00<?, ?it/s]

In [13]:
best_stoch_design = stoch_results[np.argmin([r['total'] for r in stoch_results])]
print(fmt_design_results(best_stoch_design))

Parameter         Unit      Value
----------------  ------  -------
Solar capacity    kWp       750
Battery capacity  kWh       600
Total cost        £k/yr     151.4
CAPEX             £k/yr      98.2
OPEX              £k/yr      53.1


Using a Stochastic Program for design

In [14]:
SP_results = run_model(None,None,solar_year=list(range(2012,2018)))
print(fmt_design_results(SP_results))

Parameter         Unit      Value
----------------  ------  -------
Solar capacity    kWp       865.5
Battery capacity  kWh       521.1
Total cost        £k/yr     141.9
CAPEX             £k/yr     101.4
OPEX              £k/yr      40.6


In [15]:
design_improvement = best_stoch_design['total'] - SP_results['total']
print("Using the SP for design reduces costs by "\
      f"£{(design_improvement)/1e3:,.1f}k/yr "\
      f"({round((design_improvement) / best_stoch_design['total'] * 100, 1)})% "\
        "on average")

Using the SP for design reduces costs by £9.4k/yr (6.2)% on average


The Value of Stochastic Solution (VSS)

In [16]:
try:
    run_model(LP_results['solar_capacity'],LP_results['battery_capacity'],solar_year=[2015])
except Exception as e:
    print(f"Encountered error: {e}")

Optimization potentially failed: 
Status: warning
Termination condition: infeasible_or_unbounded
Solution: 0 primals, 0 duals
Objective: nan
Solver model: available
Solver message: 4



Encountered error: Underlying model not optimized.


Hmmm, the deterministic LP leads to an infeasible design! This is clearly an issue. It's happening because the solar capacity is too high and causes the grid connection capacity to be exceeded in years with high solar generation

Let's tweak the design a little bit so it's valid (feasible), and see how close it is cost-wise to the SP solution

In [17]:
LP_avg_cost = run_model(
    LP_results['solar_capacity']*0.95,
    LP_results['battery_capacity'],
    solar_year=solar_years
    )['total']

vss = (LP_avg_cost - SP_results['total'])
print(f"Value of stochastic solution (VSS): £{round((vss)/1e3,2)}k/yr "\
      f"({round((vss)/SP_results['total']*100,2)})%")

Value of stochastic solution (VSS): £0.9k/yr (0.63)%


Now let's repeat the Stochastic optimisation but considering lots of uncertainties, as it doesn't increase the computational cost

In [18]:
nsamples = 25
scenarios = {
    'solar_year': random.choice(solar_years, nsamples),
    'load_year': random.choice(load_years, nsamples),
    'mean_load': stats.truncnorm.rvs(-2, 2, loc=100, scale=10, size=nsamples),
    'battery_efficiency': stats.truncnorm.rvs(-2, 2, loc=0.95, scale=0.05, size=nsamples),
    'battery_cost': stats.truncnorm.rvs(-2, 2, loc=70, scale=5, size=nsamples),
}

In [19]:
stoch2_results = [run_model(*design, **scenarios) for design in tqdm(design_options, desc='Assessing designs')]

Assessing designs:   0%|          | 0/5 [00:00<?, ?it/s]

In [20]:
print([round(r['capex']/1e3,1) for r in stoch2_results])
print([round(r['opex']/1e3,1) for r in stoch2_results])
print([round(r['total']/1e3,1) for r in stoch2_results])

[65.6, 79.6, 87.1, 98.4, 112.4]
[93.6, 83.2, 67.9, 45.9, 37.0]
[159.2, 162.8, 155.0, 144.3, 149.3]


In [21]:
best_stoch2_design = stoch2_results[np.argmin([r['total'] for r in stoch2_results])]
print(fmt_design_results(best_stoch2_design))

Parameter         Unit      Value
----------------  ------  -------
Solar capacity    kWp       750
Battery capacity  kWh       600
Total cost        £k/yr     144.3
CAPEX             £k/yr      98.4
OPEX              £k/yr      45.9


In [22]:
SP_results2 = run_model(
    solar_capacity=None,
    battery_capacity=None,
    solar_year=scenarios['solar_year'],
    load_year=scenarios['load_year'],
    mean_load=scenarios['mean_load'],
    battery_efficiency=scenarios['battery_efficiency'],
    battery_cost=scenarios['battery_cost']
)

In [23]:
print(fmt_design_results(SP_results2))

Parameter         Unit      Value
----------------  ------  -------
Solar capacity    kWp       791.9
Battery capacity  kWh       457.2
Total cost        £k/yr     139.1
CAPEX             £k/yr      91.5
OPEX              £k/yr      47.6


In [24]:
design_improvement = best_stoch2_design['total'] - SP_results2['total']
print("Using the SP for design reduces costs by "\
      f"£{(design_improvement)/1e3:,.1f}k/yr "\
      f"({round((design_improvement) / best_stoch2_design['total'] * 100, 1)})% "\
        "on average")

Using the SP for design reduces costs by £5.2k/yr (3.6)% on average


In [25]:
# again, the LP solution is infeasible, so let's tweak it a bit
LP_avg_cost2 = run_model(
    LP_results['solar_capacity']*0.9,
    LP_results['battery_capacity'],
    **scenarios
    )['total']

vss2 = (LP_avg_cost2 - SP_results2['total'])
print(f"Value of stochastic solution (VSS): £{round((vss)/1e3,2)}k/yr "\
      f"({round((vss)/SP_results2['total']*100,2)})%")

Value of stochastic solution (VSS): £0.9k/yr (0.65)%
